Reference: https://becominghuman.ai/local-binary-pattern-features-for-texture-classification-d0dfd86ebf29

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install scikit-multilearn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
!unzip /content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset.zip -d /content/cedar_dataset

Archive:  /content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset.zip
   creating: /content/cedar_dataset/full_org/
  inflating: /content/cedar_dataset/full_org/original_29_18.png  
  inflating: /content/cedar_dataset/full_org/original_2_6.png  
  inflating: /content/cedar_dataset/full_org/original_27_21.png  
  inflating: /content/cedar_dataset/full_org/original_29_24.png  
  inflating: /content/cedar_dataset/full_org/original_20_14.png  
  inflating: /content/cedar_dataset/full_org/original_54_18.png  
  inflating: /content/cedar_dataset/full_org/original_52_1.png  
  inflating: /content/cedar_dataset/full_org/original_17_1.png  
  inflating: /content/cedar_dataset/full_org/original_22_8.png  
  inflating: /content/cedar_dataset/full_org/original_53_11.png  
  inflating: /content/cedar_dataset/full_org/original_31_2.png  
  inflating: /content/cedar_dataset/full_org/original_54_24.png  
  inflating: /content/cedar_dataset/full_org/original_37_24.png  
  inflating: /content/ce

In [ ]:
!ls

cedar_dataset  drive  sample_data


In [ ]:
!ls cedar_dataset

full_forg  full_org  __MACOSX  Readme.txt


In [ ]:
!ls cedar_dataset/full_forg

forgeries_10_10.png  forgeries_26_22.png  forgeries_43_11.png
forgeries_10_11.png  forgeries_26_23.png  forgeries_43_12.png
forgeries_10_12.png  forgeries_26_24.png  forgeries_43_13.png
forgeries_10_13.png  forgeries_26_2.png   forgeries_43_14.png
forgeries_10_14.png  forgeries_26_3.png   forgeries_43_15.png
forgeries_10_15.png  forgeries_26_4.png   forgeries_43_16.png
forgeries_10_16.png  forgeries_26_5.png   forgeries_43_17.png
forgeries_10_17.png  forgeries_26_6.png   forgeries_43_18.png
forgeries_10_18.png  forgeries_26_7.png   forgeries_43_19.png
forgeries_10_19.png  forgeries_26_8.png   forgeries_43_1.png
forgeries_10_1.png   forgeries_26_9.png   forgeries_43_20.png
forgeries_10_20.png  forgeries_2_6.png	  forgeries_43_21.png
forgeries_10_21.png  forgeries_27_10.png  forgeries_43_22.png
forgeries_10_22.png  forgeries_27_11.png  forgeries_43_23.png
forgeries_10_23.png  forgeries_27_12.png  forgeries_43_24.png
forgeries_10_24.png  forgeries_27_13.png  forgeries_43_2.png
forgeries_1

In [ ]:
def extract_signers():
    X_train, X_test, y_train, y_test = [],[],[],[]

    for file_path in os.listdir("cedar_dataset/full_org"):
      if len(file_path.split("_")) == 3:
        tag = file_path.split("_")[0]
        id = file_path.split("_")[1]
        sign_no = file_path.split("_")[2].split(".")[0]

        if int(id) <= 20:
          X_train.append(file_path)
          y_train.append(tag)

        elif int(id) <=26:
          X_test.append(file_path)
          y_test.append(tag)

    for file_path in os.listdir("cedar_dataset/full_forg"):
      if len(file_path.split("_")) == 3:
        tag = file_path.split("_")[0]
        id = file_path.split("_")[1]
        sign_no = file_path.split("_")[2].split(".")[0]

        if int(id) <= 20:
          X_train.append(file_path)
          y_train.append(tag)

        elif int(id) <=26:
          X_test.append(file_path)
          y_test.append(tag)


    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = extract_signers()

In [ ]:
len(X_train)

960

In [ ]:
len(X_test)

288

In [ ]:

train_df = pd.DataFrame(zip(X_train, y_train))
train_df.columns = ["filename", "label"]
test_df = pd.DataFrame(zip(X_test, y_test))
test_df.columns = ["filename", "label"]

In [ ]:
train_df

,filename,label
0,original_19_2.png,original
1,original_6_12.png,original
2,original_17_3.png,original
3,original_16_1.png,original
4,original_3_9.png,original
...,...,...
955,forgeries_15_21.png,forgeries
956,forgeries_6_23.png,forgeries
957,forgeries_14_3.png,forgeries
958,forgeries_9_11.png,forgeries


In [ ]:
test_df

,filename,label
0,original_26_14.png,original
1,original_25_15.png,original
2,original_23_5.png,original
3,original_21_6.png,original
4,original_21_12.png,original
...,...,...
283,forgeries_21_9.png,forgeries
284,forgeries_22_8.png,forgeries
285,forgeries_24_13.png,forgeries
286,forgeries_25_12.png,forgeries


In [ ]:
def label_encoding(label):
  if label == "original":
    return 1
  elif label == "forgeries":
    return 0

In [ ]:
def extract_signer_id(file_path):
  return int(file_path.split("_")[1])

In [ ]:
train_df["label"] = train_df["label"].apply(label_encoding)

In [ ]:
train_df

,filename,label
0,original_19_2.png,1
1,original_6_12.png,1
2,original_17_3.png,1
3,original_16_1.png,1
4,original_3_9.png,1
...,...,...
955,forgeries_15_21.png,0
956,forgeries_6_23.png,0
957,forgeries_14_3.png,0
958,forgeries_9_11.png,0


In [ ]:
train_df["signer"] = train_df["filename"]
test_df["signer"] = test_df["filename"]

In [ ]:
train_df["signer"] = train_df["signer"].apply(extract_signer_id)
test_df["signer"] = test_df["signer"].apply(extract_signer_id)

In [ ]:
train_df

,filename,label,signer
0,original_19_2.png,1,19
1,original_6_12.png,1,6
2,original_17_3.png,1,17
3,original_16_1.png,1,16
4,original_3_9.png,1,3
...,...,...,...
955,forgeries_15_21.png,0,15
956,forgeries_6_23.png,0,6
957,forgeries_14_3.png,0,14
958,forgeries_9_11.png,0,9


In [ ]:
test_df

,filename,label,signer
0,original_21_16.png,original,21
1,original_24_20.png,original,24
2,original_21_10.png,original,21
3,original_24_2.png,original,24
4,original_21_21.png,original,21
...,...,...,...
283,forgeries_25_20.png,forgeries,25
284,forgeries_24_4.png,forgeries,24
285,forgeries_26_18.png,forgeries,26
286,forgeries_23_19.png,forgeries,23


In [ ]:
# signer_14 = train_df[train_df["signer"]==14]

In [ ]:
values_to_extract = [14, 18,9]
signers_df = train_df[train_df['signer'].isin(values_to_extract)]

In [ ]:
# signer_18 = train_df[train_df["signer"]==18]

In [ ]:
signers_df

,filename,label,signer
6,original_18_1.png,1,18
10,original_9_12.png,1,9
14,original_14_9.png,1,14
15,original_18_15.png,1,18
17,original_14_20.png,1,14
...,...,...,...
915,forgeries_14_22.png,0,14
920,forgeries_9_2.png,0,9
940,forgeries_9_20.png,0,9
957,forgeries_14_3.png,0,14


In [ ]:
def extract_multiclass_label(filename):
  genuine_forged = filename.split("_")[0]
  signer  = filename.split("_")[1]

  return signer, genuine_forged


In [ ]:
signers_df["tag"] = signers_df["filename"]
signers_df["tag"] = signers_df["tag"].apply(extract_multiclass_label)

<ipython-input-33-2b2c48da9f9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signers_df["tag"] = signers_df["filename"]
<ipython-input-33-2b2c48da9f9d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signers_df["tag"] = signers_df["tag"].apply(extract_multiclass_label)


In [ ]:
signers_df

,filename,label,signer,tag
6,original_18_1.png,1,18,"(18, original)"
10,original_9_12.png,1,9,"(9, original)"
14,original_14_9.png,1,14,"(14, original)"
15,original_18_15.png,1,18,"(18, original)"
17,original_14_20.png,1,14,"(14, original)"
...,...,...,...,...
915,forgeries_14_22.png,0,14,"(14, forgeries)"
920,forgeries_9_2.png,0,9,"(9, forgeries)"
940,forgeries_9_20.png,0,9,"(9, forgeries)"
957,forgeries_14_3.png,0,14,"(14, forgeries)"


In [ ]:
tag_list = list()

for tags in signers_df["tag"].unique():
    for tag in tags:
        if tag not in tag_list:
            tag_list.append(tag)

print(tag_list)

['18', 'original', '9', '14', 'forgeries']


In [ ]:
def one_hot_encoding(tag_list, df, tags):
    for tag in tag_list:
        columns = list()
        for line_tag in df[tags]:
            if tag in line_tag:
                columns.append(1)
            else:
                columns.append(0)

        df[tag] = columns


In [ ]:
one_hot_encoding(tag_list, signers_df, "tag")
signers_df.head()

<ipython-input-37-7beb9971d924>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[tag] = columns
<ipython-input-37-7beb9971d924>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[tag] = columns
<ipython-input-37-7beb9971d924>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,filename,label,signer,tag,18,original,9,14,forgeries
6,original_18_1.png,1,18,"(18, original)",1,1,0,0,0
10,original_9_12.png,1,9,"(9, original)",0,1,1,0,0
14,original_14_9.png,1,14,"(14, original)",0,1,0,1,0
15,original_18_15.png,1,18,"(18, original)",1,1,0,0,0
17,original_14_20.png,1,14,"(14, original)",0,1,0,1,0


In [ ]:
# path = "/content/cedar_dataset/full_forg/"
# filename = "forgeries_10_1.png"
# image = cv2.imread(os.path.join(path, filename))

# image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Perform Otsu's thresholding to get the optimal threshold value
# _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# # Save the binarized image to the output path
# # cv2.imwrite(output_image_path, thresholded_image)


In [ ]:
# cv2_imshow(thresholded_image)

In [ ]:
def preprocess_image(filename):
  if filename.split("_")[0] == "original":
    path = "/content/cedar_dataset/full_org/"

    if not os.path.exists("/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_org/"):
      os.makedirs("/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_org/")
    output_path = "/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_org/"

  elif filename.split("_")[0] == "forgeries":
    path = "/content/cedar_dataset/full_forg/"

    if not os.path.exists("/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_forg/"):
      os.makedirs("/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_forg/")
    output_path = "/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_forg/"

  image = cv2.imread(os.path.join(path, filename))

  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

  image = cv2.resize(image, dsize=(600,300))

  # Perform Otsu's thresholding to get the optimal threshold value
  _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  # print(image.shape)
  output_image_path = os.path.join(output_path, filename)
  # Save the binarized image to the output path
  cv2.imwrite(output_image_path, thresholded_image)




In [ ]:
#Binarize all images
for filename in train_df["filename"]:
  preprocess_image(filename)


In [ ]:
train_df

,filename,label,signer
0,original_1_21.png,1,1
1,original_9_17.png,1,9
2,original_9_12.png,1,9
3,original_11_5.png,1,11
4,original_18_12.png,1,18
...,...,...,...
955,forgeries_13_8.png,0,13
956,forgeries_8_5.png,0,8
957,forgeries_8_2.png,0,8
958,forgeries_8_13.png,0,8


In [ ]:

def get_images(df):
   counter = 0
   class_images = []
   for filename in df["filename"]:
       if filename.split("_")[0] == "original":
           path = "/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_org/"
       elif filename.split("_")[0] == "forgeries":
           path = "/content/drive/MyDrive/SignatureVerificationCylab/cedar_dataset_preprocessed/full_forg/"

       image = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)

      #  print(image.shape)

       class_images.append(image)


   return np.array(class_images)





In [ ]:
signers_df

,filename,label,signer,tag,18,original,9,14,forgeries
6,original_18_1.png,1,18,"(18, original)",1,1,0,0,0
10,original_9_12.png,1,9,"(9, original)",0,1,1,0,0
14,original_14_9.png,1,14,"(14, original)",0,1,0,1,0
15,original_18_15.png,1,18,"(18, original)",1,1,0,0,0
17,original_14_20.png,1,14,"(14, original)",0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
915,forgeries_14_22.png,0,14,"(14, forgeries)",0,0,0,1,1
920,forgeries_9_2.png,0,9,"(9, forgeries)",0,0,1,0,1
940,forgeries_9_20.png,0,9,"(9, forgeries)",0,0,1,0,1
957,forgeries_14_3.png,0,14,"(14, forgeries)",0,0,0,1,1


In [ ]:
list_zip_tags = list(zip(signers_df["18"], signers_df["original"], signers_df["9"], signers_df["14"], signers_df["forgeries"]))

signers_df["all_tags"] = list_zip_tags
signers_df.head()

<ipython-input-68-5ec99542754d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signers_df["all_tags"] = list_zip_tags


,filename,label,signer,tag,18,original,9,14,forgeries,all_tags
6,original_18_1.png,1,18,"(18, original)",1,1,0,0,0,"(1, 1, 0, 0, 0)"
10,original_9_12.png,1,9,"(9, original)",0,1,1,0,0,"(0, 1, 1, 0, 0)"
14,original_14_9.png,1,14,"(14, original)",0,1,0,1,0,"(0, 1, 0, 1, 0)"
15,original_18_15.png,1,18,"(18, original)",1,1,0,0,0,"(1, 1, 0, 0, 0)"
17,original_14_20.png,1,14,"(14, original)",0,1,0,1,0,"(0, 1, 0, 1, 0)"


In [ ]:
signers_preprocessed= get_images(signers_df)

In [ ]:
signers_preprocessed

array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       ...,

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255,   0],
        [255, 255, 255, ..., 255, 255, 255

In [ ]:
signers_labels = signers_df[tag_list]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(signers_preprocessed, signers_labels, test_size=0.3)

In [ ]:
def extract_lbp(images):
    lbps = []
    for image in tqdm(images):
        lbp = local_binary_pattern(image, P=8, R=1)
        lbps.append(lbp)

    return np.array(lbps)

In [ ]:
X_train_lbp = extract_lbp(X_train)
X_test_lbp = extract_lbp(X_test)

100%|██████████| 44/44 [00:01<00:00, 32.61it/s]


In [ ]:
def create_histograms(images, sub_images_num, bins_per_sub_images):
    all_histograms = []
    for image in tqdm(images):
        grid = np.arange(0, image.shape[1]+1, image.shape[1]//sub_images_num)

        sub_image_histograms = []

        for i in range(1, len(grid)):
            for j in range(1, len(grid)):
                sub_image = image[grid[i-1]:grid[i], grid[j-1]:grid[j]]

                sub_image_histogram = np.histogram(sub_image, bins=bins_per_sub_images)[0]
                sub_image_histograms.append(sub_image_histogram)

        histogram = np.array(sub_image_histograms).flatten()
        all_histograms.append(histogram)

    return np.array(all_histograms)


In [ ]:
X_train_hist = create_histograms(X_train_lbp, sub_images_num=3, bins_per_sub_images=64)
X_test_hist = create_histograms(X_test_lbp, sub_images_num=3, bins_per_sub_images=64)

print('X_train_hist\t', X_train_hist.shape)
print('X_test_hist\t', X_test_hist.shape)

100%|██████████| 44/44 [00:00<00:00, 225.03it/s]

X_train_hist	 (100, 576)
X_test_hist	 (44, 576)


In [ ]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    # Create an Instance
    clf = mlb_estimator(model)
    n_samples, height, width = X_train.shape
    x_train_reshaped = X_train.reshape(n_samples, height*width)

    clf.fit(x_train_reshaped,ytrain)

    n_samples, height, width = X_test.shape
    x_test_reshaped = X_test.reshape(n_samples, height*width)

    # Predict
    clf_predictions = clf.predict(x_test_reshaped)
    # Check For Accuracy
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result, clf_predictions

In [ ]:
print(len(X_train))

100


In [ ]:
print(len(y_train))

100


In [ ]:
print(len(y_train))

100


In [ ]:
X_train.shape

(100, 300, 600)

Classifier chain

In [ ]:
clf_chain_model, pred = build_model(SVC(),ClassifierChain,X_train,y_train,X_test,y_test)
clf_chain_model

{'accuracy:': 0.045454545454545456, 'hamming_score': 0.36818181818181817}

In [ ]:
print(pred.toarray())

[[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 1]]


In [ ]:
print(y_test)

     18  original  9  14  forgeries
777   0         0  0   1          1
565   1         0  0   0          1
855   0         0  1   0          1
279   1         1  0   0          0
545   1         0  0   0          1
367   1         1  0   0          0
781   0         0  1   0          1
353   0         1  0   1          0
422   0         1  1   0          0
254   1         1  0   0          0
226   0         1  0   1          0
670   0         0  1   0          1
248   0         1  1   0          0
17    0         1  0   1          0
432   0         1  1   0          0
624   1         0  0   0          1
348   1         1  0   0          0
368   0         1  0   1          0
251   0         1  1   0          0
474   0         1  0   1          0
537   1         0  0   0          1
62    1         1  0   0          0
240   0         1  1   0          0
909   0         0  1   0          1
134   0         1  1   0          0
364   0         1  0   1          0
18    0         1  1   0    

In [ ]:
clf_chain_model, pred = build_model(KNeighborsClassifier(n_neighbors=5),ClassifierChain,X_train,y_train,X_test,y_test)
clf_chain_model

{'accuracy:': 0.22727272727272727, 'hamming_score': 0.42727272727272725}

In [ ]:
clf_chain_model, pred = build_model(RandomForestClassifier(n_estimators=100, random_state=42),ClassifierChain,X_train,y_train,X_test,y_test)
clf_chain_model

{'accuracy:': 0.22727272727272727, 'hamming_score': 0.2909090909090909}

Label Powerset

In [ ]:
clf_labelP_model, pred = build_model(SVC(),LabelPowerset,X_train,y_train,X_test,y_test)
clf_labelP_model

{'accuracy:': 0.38636363636363635, 'hamming_score': 0.3}

In [ ]:
clf_labelP_model, pred = build_model(KNeighborsClassifier(n_neighbors=5),LabelPowerset,X_train,y_train,X_test,y_test)
clf_labelP_model

{'accuracy:': 0.20454545454545456, 'hamming_score': 0.43636363636363634}

In [ ]:
clf_labelP_model, pred = build_model(RandomForestClassifier(n_estimators=100, random_state=42),LabelPowerset,X_train,y_train,X_test,y_test)
clf_labelP_model

{'accuracy:': 0.4090909090909091, 'hamming_score': 0.2909090909090909}